<a href="https://colab.research.google.com/github/ngonhan2k5/cs-ml/blob/master/proj1/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Prepare dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [33]:
#load data
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', skipinitialspace=True, na_values="?")
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 
                     'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 
                     'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
#df.describe()
df.sample(4)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
8692,45,Private,226246,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,50,United-States,<=50K
10761,45,Self-emp-inc,185497,Bachelors,13,Married-civ-spouse,Exec-managerial,Wife,Asian-Pac-Islander,Female,0,0,70,NaN,<=50K
26900,42,Private,86185,Some-college,10,Widowed,Exec-managerial,Not-in-family,Amer-Indian-Eskimo,Female,0,0,40,United-States,<=50K
6737,36,Private,218490,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,35,United-States,<=50K


In [35]:
#remove records missing values
no_missing_df= df.drop(df[df.native_country.isnull()].index)
no_missing_df= no_missing_df.drop(no_missing_df[no_missing_df.workclass.isnull()].index)
no_missing_df= no_missing_df.drop(no_missing_df[no_missing_df.occupation.isnull()].index)
no_missing_df.isnull().sum().max() #just checking that there's no missing data missing...

0

In [36]:
nomissing_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income'],
      dtype='object')

In [37]:
#LabelEncoder
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
no_missing_df['workclass_num']= le.fit_transform(no_missing_df['workclass'])
no_missing_df['marital_status_num']= le.fit_transform(no_missing_df['marital_status'])
no_missing_df['occupation_num']= le.fit_transform(no_missing_df['occupation'])
no_missing_df['relationship_num']= le.fit_transform(no_missing_df['relationship'])
no_missing_df['race_num']= le.fit_transform(no_missing_df['race'])
no_missing_df['sex_num']= le.fit_transform(no_missing_df['sex'])
no_missing_df['native_country_num']= le.fit_transform(no_missing_df['native_country'])
no_missing_df['income_num']= le.fit_transform(no_missing_df['income'])

no_missing_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'workclass_num', 'marital_status_num', 'occupation_num',
       'relationship_num', 'race_num', 'sex_num', 'native_country_num',
       'income_num'],
      dtype='object')

In [39]:
#remove the categorical features
neccessary_columns_df= no_missing_df.drop("workclass", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("education", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("marital_status", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("occupation", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("relationship", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("race", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("sex", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("native_country", axis='columns')
neccessary_columns_df= neccessary_columns_df.drop("income", axis='columns')

#remove the unneccessary columns
neccessary_columns_df= neccessary_columns_df.drop("fnlwgt", axis='columns')

neccessary_columns_df.columns

Index(['age', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week', 'workclass_num', 'marital_status_num',
       'occupation_num', 'relationship_num', 'race_num', 'sex_num',
       'native_country_num', 'income_num'],
      dtype='object')

In [46]:
#separate input and target
input_df= neccessary_columns_df.drop('income_num', axis='columns')
target_df= neccessary_columns_df['income_num']
input_df.sample(2)
target_df.sample(2)

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass_num,marital_status_num,occupation_num,relationship_num,race_num,sex_num,native_country_num
6010,34,10,7298,0,40,2,2,3,0,4,1,38
22273,38,9,0,0,35,2,0,7,1,4,0,38


16610    0
17526    0
Name: income_num, dtype: int64

# 2. Split data into Training and Testing

In [48]:
#Split data into Training and Testing dataframe
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_df, target_df, test_size=0.3, random_state=0)
print("Training Data:", X_train.shape, y_train.shape)
print("Testing Data :", X_test.shape, y_test.shape)

Training Data: (21112, 12) (21112,)
Testing Data : (9049, 12) (9049,)


# Tune hyper-parameters manually

# GridSearchCV to tune hyper-parameters

In [88]:
from sklearn.model_selection import GridSearchCV
def get_the_GridSearcCV(model, param_dist, X_train, X_test, y_train, y_test):
  gs= GridSearchCV(model, param_dist)
  gs.fit(X_train, y_train)
  print('Tuning Hyper-Parameters:')
  print('  - The best parameters: ', gs.best_params_)
  print('  - The best score: ', gs.best_score_)
  testing_score= gs.score(X_test, y_test)
  print('Testing score:', testing_score)
  return gs

In [111]:
#Tuning hyper-parameters by using GridSearchCV
from sklearn.tree import DecisionTreeClassifier
print('*Decision Tree')
param_dist= {'criterion': ['gini','entropy'],
             'max_depth': [1,2,3,4,5,6,7,8,9],
             'min_samples_leaf': [2,3,4,5,6],
             'min_samples_split': [4,5,6,7,8,9,10,11,12,13,14,15]}
gs= get_the_GridSearcCV(DecisionTreeClassifier(), param_dist,  X_train, X_test, y_train, y_test)

*Decision Tree
Tuning Hyper-Parameters:
  - The best parameters:  {'criterion': 'gini', 'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 5}
  - The best score:  0.8531168726716263
Testing score: 0.8455077909161234


In [106]:
y_predict= gs.predict([X_test.iloc[0], X_test.iloc[1], X_test.iloc[2], X_test.iloc[8], X_test.iloc[12], X_test.iloc[24],])
print('Predicted result:', y_predict, ' -- Expected result: [', y_test.iloc[0], y_test.iloc[1], y_test.iloc[2], y_test.iloc[8], y_test.iloc[12], y_test.iloc[24], ']')

Predicted result: [0 0 0 0 1 1]  -- Expected result: [ 0 0 0 1 1 1 ]


In [110]:
y_predict= gs.predict(X_test)

from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test, y_predict)
confusion_df= pd.DataFrame(cm)
confusion_df

,0,1
0,6473,397
1,992,1187
